In [22]:
import pandas as pd
import mysql.connector
import sqlalchemy

In [23]:
cnx = mysql.connector.connect(user='root', password='password', 
                              host='127.0.0.1', database='otus_db', port='3306')

In [24]:
cur = cnx.cursor()

In [25]:
группировки с ипользованием CASE, HAVING, ROLLUP, GROUPING() :
для магазина к предыдущему списку продуктов добавить максимальную и минимальную цену и кол-во предложений
сделать выборку показывающую самый дорогой и самый дешевый товар в каждой категории
сделать rollup с количеством товаров по категориям

SyntaxError: invalid syntax (1691857623.py, line 1)

#### Создадим таблицу products и наполним данными 
##### title - наименование товара
##### category - категория товара
##### price - цена товара
##### rating - рейтинг товара
##### sales - кол-во продаж товара
##### sales - кол-во продаж товара
##### status - статус наличия товара

In [109]:
cur.execute('''
            DROP TABLE IF EXISTS products;
            ''')

cur.execute('''
            CREATE TABLE products (
                title VARCHAR(32) NOT NULL,
                category VARCHAR(32),
                price INT,
                rating INT,
                sales INT,
                status VARCHAR(32) NOT NULL,
                PRIMARY KEY (title));
            ''')

cur.execute('''
            INSERT INTO products
                (title, category, price, rating, sales, status)
            VALUES
                ('Агдам', 'Напитки', 150, 2, 4, 'В наличии'),
                ('Килька', 'Консервы', 45, 4, 10, 'Распродан'),
                ('Оливки', 'Консервы', 250, 5, 24, 'Распродан'),
                ('Текила', 'Напитки', 3000, 5, 23, 'В наличии'),
                ('Шмурдяк', 'Напитки', 120, 1, 15, 'Распродан'),
                ('Арахис', 'Орехи', 250, 5, 12, 'Распродан'),
                ('Фисташки', 'Орехи', 450, 5, 4, 'В наличии');
            ''')
query = f'''
            SELECT * from products;
        '''
df = pd.read_sql_query(query, cnx)
df

/Users/denis/opt/anaconda3/lib/python3.9/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,title,category,price,rating,sales,status
0,Агдам,Напитки,150,2,4,В наличии
1,Арахис,Орехи,250,5,12,Распродан
2,Килька,Консервы,45,4,10,Распродан
3,Оливки,Консервы,250,5,24,Распродан
4,Текила,Напитки,3000,5,23,В наличии
5,Фисташки,Орехи,450,5,4,В наличии
6,Шмурдяк,Напитки,120,1,15,Распродан


#### Группировки с ипользованием CASE
#####  Наличие товаров каждой категории

In [53]:
query = f'''
          SELECT
          category,
          SUM( CASE WHEN status = 'В наличии'
                THEN 1
                ELSE 0
                END ) AS total
        FROM products
        GROUP BY category;
        '''
df = pd.read_sql_query(query, cnx)
df

/Users/denis/opt/anaconda3/lib/python3.9/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,category,total
0,Напитки,2.0
1,Орехи,1.0
2,Консервы,0.0


#### Группировки с ипользованием HAVING
#### Вывести категории товары которых были проданы более 20 раз

In [54]:
query = f'''
          SELECT
          category,
          SUM(sales) AS total
        FROM products
        GROUP BY category
        HAVING total > 20;
        '''
df = pd.read_sql_query(query, cnx)
df

/Users/denis/opt/anaconda3/lib/python3.9/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,category,total
0,Напитки,42.0
1,Консервы,34.0


#### Группировки с ипользованием ROLLUP,  GROUPING()
#####  Вывести кол-во продаж каждой категории и общий итог 

In [55]:
query = f'''
        SELECT
          IF(GROUPING(category), 'ИТОГО', category) AS category,
          SUM(sales) AS total
        FROM products
        GROUP BY category WITH ROLLUP;
        '''
df = pd.read_sql_query(query, cnx)
df

/Users/denis/opt/anaconda3/lib/python3.9/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,category,total
0,Консервы,34.0
1,Напитки,42.0
2,Орехи,16.0
3,ИТОГО,92.0


#### для магазина к предыдущему списку продуктов добавить максимальную и минимальную цену и кол-во предложений

In [69]:
query = f'''
        SELECT
          category,
          title,
          price,
          sales,
          min(price) over (partition by category) as min_price_in_category,
          max(price) over (partition by category) as max_price_in_category,
          sum(sales) over (partition by category) as sales_in_category
        FROM products;
        '''
df = pd.read_sql_query(query, cnx)
df

/Users/denis/opt/anaconda3/lib/python3.9/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,category,title,price,sales,min_price_in_category,max_price_in_category,sales_in_category
0,Консервы,Килька,45,10,45,250,34.0
1,Консервы,Оливки,250,24,45,250,34.0
2,Напитки,Агдам,150,4,120,3000,42.0
3,Напитки,Текила,3000,23,120,3000,42.0
4,Напитки,Шмурдяк,120,15,120,3000,42.0
5,Орехи,Арахис,250,12,250,450,16.0
6,Орехи,Фисташки,450,4,250,450,16.0


#### сделать выборку показывающую самый дорогой и самый дешевый товар в каждой категории

In [110]:
query = f'''
with min_max as (
        SELECT
          category,
          title,
          price,
          min(price) over (partition by category) as min_price,
          max(price) over (partition by category) as max_price
        FROM products)
        
select 
        l.category,
        l.title as min_title,
        r.title as max_title
        from min_max l
        join min_max r
        on l.category = r.category
        and r.max_price = r.price
        where l.min_price = l.price;
        '''
df = pd.read_sql_query(query, cnx)
df

/Users/denis/opt/anaconda3/lib/python3.9/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,category,min_title,max_title
0,Консервы,Килька,Оливки
1,Напитки,Шмурдяк,Текила
2,Орехи,Арахис,Фисташки


#### сделать rollup с количеством товаров по категориям

In [112]:
query = f'''
        SELECT
          IF(GROUPING(category), 'ИТОГО', category) AS category,
          SUM( CASE WHEN status = 'В наличии'
                THEN 1
                ELSE 0
                END ) AS total
        FROM products
        GROUP BY category WITH ROLLUP;
        '''
df = pd.read_sql_query(query, cnx)
df

/Users/denis/opt/anaconda3/lib/python3.9/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,category,total
0,Консервы,0.0
1,Напитки,2.0
2,Орехи,1.0
3,ИТОГО,3.0
